In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import time
from dotenv import load_dotenv
import os
import requests
import pprint as ppp

In [4]:
# Prueba de Conexion por usuarios (?, creo)
url_1 = "https://twitter135.p.rapidapi.com/AutoComplete/"

querystring = {"q":"Elon"}

headers = {
	"x-rapidapi-key": "2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a",
	"x-rapidapi-host": "twitter135.p.rapidapi.com"
}

response_1 = requests.get(url_1, headers=headers, params=querystring)

print(response_1.json())

{'num_results': 13, 'users': [{'id': 44196397, 'id_str': '44196397', 'verified': False, 'ext_is_blue_verified': True, 'badges': [{'badge_url': 'https://pbs.twimg.com/profile_images/1683899100922511378/5lY42eHs_bigger.jpg', 'badge_type': 'BusinessLabel', 'description': 'X'}], 'is_dm_able': False, 'is_secret_dm_able': False, 'is_blocked': False, 'can_media_tag': False, 'name': 'Elon Musk', 'screen_name': 'elonmusk', 'profile_image_url': 'http://pbs.twimg.com/profile_images/1815749056821346304/jS8I28PL_normal.jpg', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1815749056821346304/jS8I28PL_normal.jpg', 'location': '', 'is_protected': False, 'rounded_score': 0, 'social_proof': 0, 'connecting_user_count': 0, 'connecting_user_ids': [], 'social_proofs_ordered': [], 'social_context': {'following': False, 'followed_by': False}, 'tokens': [], 'inline': False}, {'id': 1382209054999646212, 'id_str': '1382209054999646212', 'verified': False, 'ext_is_blue_verified': True, 'badges':

In [5]:
ppp.pprint(response_1.json())

{'completed_in': 0.0,
 'events': [],
 'hashtags': [],
 'lists': [],
 'num_results': 13,
 'oneclick': [],
 'ordered_sections': [],
 'query': 'Elon',
 'topics': [{'inline': False,
             'rounded_score': 0,
             'tokens': [],
             'topic': 'elonmusk'},
            {'inline': False,
             'rounded_score': 0,
             'tokens': [],
             'topic': '#Elon_Musk'},
            {'inline': False,
             'rounded_score': 0,
             'tokens': [],
             'topic': 'elon block function'}],
 'users': [{'badges': [{'badge_type': 'BusinessLabel',
                        'badge_url': 'https://pbs.twimg.com/profile_images/1683899100922511378/5lY42eHs_bigger.jpg',
                        'description': 'X'}],
            'can_media_tag': False,
            'connecting_user_count': 0,
            'connecting_user_ids': [],
            'ext_is_blue_verified': True,
            'id': 44196397,
            'id_str': '44196397',
            'inline': Fals

In [8]:
# Extraccion de datos con modalidad "search"
url_2 = "https://twitter135.p.rapidapi.com/Search/"

user_search = 'Colapinto'
amount_tweets = 130

querystring = {"q":user_search,"count":amount_tweets,"type":"Top","safe_search":"false"}

headers = {
	"x-rapidapi-key": "2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a",
	"x-rapidapi-host": "twitter135.p.rapidapi.com"
}

response_2 = requests.get(url_2, headers=headers, params=querystring)

print(response_2.json())

{'data': {'search_by_raw_query': {'search_timeline': {'timeline': {'instructions': [{'type': 'TimelineAddEntries', 'entries': [{'entryId': 'toptabsrpusermodule-1839786171924742144', 'sortIndex': '1839786171924742144', 'content': {'entryType': 'TimelineTimelineModule', '__typename': 'TimelineTimelineModule', 'items': [{'entryId': 'toptabsrpusermodule-1839786171924742144-user-2994668849', 'item': {'itemContent': {'itemType': 'TimelineUser', '__typename': 'TimelineUser', 'user_results': {'result': {'__typename': 'User', 'id': 'VXNlcjoyOTk0NjY4ODQ5', 'rest_id': '2994668849', 'affiliates_highlighted_label': {}, 'has_graduated_access': True, 'is_blue_verified': True, 'profile_image_shape': 'Circle', 'legacy': {'can_dm': False, 'can_media_tag': False, 'created_at': 'Fri Jan 23 19:42:50 +0000 2015', 'default_profile': True, 'default_profile_image': False, 'description': '🇦🇷 Formula 1 driver for @williamsracing Management:@BulletSportsMgt', 'entities': {'description': {'urls': []}, 'url': {'url

In [9]:
ppp.pprint(response_2.json())

{'data': {'search_by_raw_query': {'search_timeline': {'timeline': {'instructions': [{'entries': [{'content': {'__typename': 'TimelineTimelineModule',
                                                                                                              'clientEventInfo': {'component': 'user_module',
                                                                                                                                  'element': 'module'},
                                                                                                              'displayType': 'Vertical',
                                                                                                              'entryType': 'TimelineTimelineModule',
                                                                                                              'footer': {'displayType': 'Classic',
                                                                                                           

In [11]:
# Pruebo otra api de X de rapidapi

url_3 = "https://twitter-x.p.rapidapi.com/search/"

querystring = {"query":"LLFDammit","section":"top","limit":"20"}

headers = {
	"x-rapidapi-key": "2e2c904e0cmshdbad92d97808688p1e798ajsna8b04a4dd68a",
	"x-rapidapi-host": "twitter-x.p.rapidapi.com"
}

response_3 = requests.get(url_3, headers=headers, params=querystring)

print(response_3.json())

{'data': {'search_by_raw_query': {'search_timeline': {'timeline': {'instructions': [{'type': 'TimelineAddEntries', 'entries': [{'entryId': 'tweet-1838894496899096674', 'sortIndex': '1839786770904907776', 'content': {'entryType': 'TimelineTimelineItem', '__typename': 'TimelineTimelineItem', 'itemContent': {'itemType': 'TimelineTweet', '__typename': 'TimelineTweet', 'tweet_results': {'result': {'__typename': 'Tweet', 'rest_id': '1838894496899096674', 'core': {'user_results': {'result': {'__typename': 'User', 'id': 'VXNlcjoxNzUyOTkyODU2NDU1NzQxNDQw', 'rest_id': '1752992856455741440', 'affiliates_highlighted_label': {}, 'has_graduated_access': True, 'is_blue_verified': False, 'profile_image_shape': 'Circle', 'legacy': {'can_dm': False, 'can_media_tag': True, 'created_at': 'Thu Feb 01 09:50:53 +0000 2024', 'default_profile': True, 'default_profile_image': False, 'description': '', 'entities': {'description': {'urls': []}}, 'fast_followers_count': 0, 'favourites_count': 612, 'followers_count

In [12]:
ppp.pprint(response_3.json())

{'data': {'search_by_raw_query': {'search_timeline': {'timeline': {'instructions': [{'entries': [{'content': {'__typename': 'TimelineTimelineItem',
                                                                                                              'clientEventInfo': {'component': 'result',
                                                                                                                                  'details': {'timelinesDetails': {'controllerData': 'DAACDAAFDAABDAABDAABCgABAAAAAAAAgAAAAAwAAgoAAQAAAAAAAAABCgACHT2ujUKJdeMLAAMAAAAJTExGRGFtbWl0CgAFX8xuyAPsOfgIAAYAAAABCgAHPJIQLp/AGnQAAAAAAA=='}},
                                                                                                                                  'element': 'tweet'},
                                                                                                              'entryType': 'TimelineTimelineItem',
                                                                          

In [15]:
tweets_results_3 = response_3.json()  # instancio los resultados
tweets_results_3

{'data': {'search_by_raw_query': {'search_timeline': {'timeline': {'instructions': [{'type': 'TimelineAddEntries',
       'entries': [{'entryId': 'tweet-1838894496899096674',
         'sortIndex': '1839786770904907776',
         'content': {'entryType': 'TimelineTimelineItem',
          '__typename': 'TimelineTimelineItem',
          'itemContent': {'itemType': 'TimelineTweet',
           '__typename': 'TimelineTweet',
           'tweet_results': {'result': {'__typename': 'Tweet',
             'rest_id': '1838894496899096674',
             'core': {'user_results': {'result': {'__typename': 'User',
                'id': 'VXNlcjoxNzUyOTkyODU2NDU1NzQxNDQw',
                'rest_id': '1752992856455741440',
                'affiliates_highlighted_label': {},
                'has_graduated_access': True,
                'is_blue_verified': False,
                'profile_image_shape': 'Circle',
                'legacy': {'can_dm': False,
                 'can_media_tag': True,
             

In [14]:
# Entiendo que esta es la ruta a cada twitt
entries = tweets_results_3['data']['search_by_raw_query']['search_timeline']['timeline']['instructions'][0]['entries']

# Si tengo twitts
if len(entries) > 0:
    for i, entry in enumerate(entries):
        if i >= 5:  # Que se corte cuando pase los 5
            break
        
        # Si existen los twitts accedo al contenido
        if 'content' in entry and 'itemContent' in entry['content'] and 'tweet_results' in entry['content']['itemContent']:
            tweet_result = entry['content']['itemContent']['tweet_results']['result']
            
            # Verifico si hay 'core' y 'legacy' en el tweet_result
            if 'core' in tweet_result:
                core = tweet_result['core']
                
                # si 'legacy' esta en core
                if 'legacy' in core:
                    tweet = core['legacy'].get('full_text', 'Texto no disponible')  # get para evitar KeyError
                else:
                    tweet = 'No hay legacy'  # para ver en el print e ir depurando el code
            else:
                tweet = 'No hay Core'  # Si core no esta
            
            ppp.pprint(f'''Tweet n°: {i + 1}:
                           {tweet}  # Aquí se muestra el valor del tweet
                           Fin tweet n°: {i + 1}...
                           Siguiente tweet:''')
        else:
            print(f"Entrada n°: {i + 1} no contiene un tweet válido.")
else:
    print("No hay tweets disponibles.")



('Tweet n°: 1:\n'
 '                           No hay legacy  # Aquí se muestra el valor del '
 'tweet\n'
 '                           Fin tweet n°: 1...\n'
 '                           Siguiente tweet:')
('Tweet n°: 2:\n'
 '                           No hay legacy  # Aquí se muestra el valor del '
 'tweet\n'
 '                           Fin tweet n°: 2...\n'
 '                           Siguiente tweet:')
Entrada n°: 3 no contiene un tweet válido.
Entrada n°: 4 no contiene un tweet válido.


In [ ]:
#sigo depurando el codigo porque no encuentro porque no logro acceder a 'full_text', que es lo que contiene el code